In [ ]:
import fiber
from fiber.cohort import Cohort
from fiber.condition import Patient, MRNs, Weight
from fiber.condition import Diagnosis
from fiber.condition import Measurement, Encounter, Drug
from fiber.storage import yaml as fiberyaml
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import os
from functools import reduce 
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter

In [ ]:
Case_complete = pq.read_table('Cohorts/Case/Case_complete.parquet').to_pandas()
Control_complete = pq.read_table('Cohorts/Control/Control_complete.parquet').to_pandas()
Case_15_540 = pq.read_table('Cohorts/Case/Case_filtered_15_540.parquet').to_pandas()
Control_15_540 = pq.read_table('Cohorts/Control/Control_filtered_15_540.parquet').to_pandas()

In [ ]:
Case_complete

In [ ]:
Case_complete.to_parquet("Cohorts/Case/Case_complete.parquet")

In [ ]:
Case_complete.loc[Case_complete["date_of_birth"] > 2000]

In [ ]:
Case_complete

In [ ]:
def df_to_cohort(df):
    mrns = list(df.index.values)
    condition = MRNs(mrns)
    return Cohort(condition)

In [ ]:
control = df_to_cohort(Control_complete)

In [ ]:

Control_Patients = []
Control_MRNs = list(Control_complete.index.values)
for limit in range(0, len(Control_MRNs), 500000):
    print("Begin of iteration: " +  str(limit))
    temp = Control_MRNs[limit:(limit+500000)]
    p_condition = MRNs(temp) #how to create cohort from dataframe
    cohort = Cohort(p_condition)
    print(len(cohort))
    enc = cohort.get(Patient())
    Control_Patients.append(enc)
    print("End of iteration: " +  str(limit))

In [ ]:
Control_Patients

In [ ]:
Control_all = pd.DataFrame()
for df in Control_Patients:
    Control_all = pd.concat([Control_all, df])

In [ ]:
x = Control_all.set_index("medical_record_number")

In [ ]:
x = x[["date_of_birth", "gender", "religion", "race"]]

In [ ]:
x

In [ ]:
for i in x.index:
    x.at[i, "date_of_birth"] = x.at[i, "date_of_birth"].year

In [ ]:
x.at["3916279045", "date_of_birth"]

In [ ]:
Control_complete = Control_complete.merge(x, left_index=True, right_index=True)

In [ ]:
Control_complete.to_parquet("Cohorts/Control/Control_complete.parquet")

In [ ]:
x = patients[["medical_record_number", "gender"]]

In [ ]:

x.set_index("medical_record_number", inplace=True)

In [ ]:
Case_complete = Case_complete.merge(x, left_index=True, right_index=True)

In [ ]:
Case_complete.to_parquet("Cohorts/Case/Case_complete.parquet")

In [ ]:
def get_diagram(dataframe, diagramm_type, column_name, bin_size):
    if diagramm_type=='hist':
        plt.figure(figsize=[20,8])
        plt.hist(dataframe[column_name], bins=bin_size, rwidth=4)
        plt.xticks(rotation='vertical')
    else: 
        print("diagramm type not available")

In [ ]:
get_diagram(Case_complete, "hist", "race", 100)